# Problem Set 4- ML -Preethi

In [1]:
#Loading the packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import matplotlib as mpl

In [2]:
#Reading the data
df = pd.read_csv('C:/Users/Preethi Abraham/Desktop/Brandeis Studies/Machine Learning and Data Analytics/KaggleAmes.csv')

In [3]:
# drop a bunch of useless series
bigDrop = False
if (bigDrop):
    df.drop(df.columns[[0,2,6, 7, 8,9,10,11,12,13,14,21,22,23,24,25,27,28,29,
                        32,33,35,36,39,40,42,45,47,48,49,50,52,53,55,57,60,63,
                        64,65,68,69,70,71,72,73,74,75,76,78,79]],axis=1,inplace=True)
if(not(bigDrop)):
    df.drop(df.columns[[0]],axis=1,inplace=True)

In [4]:
print(df.describe())
print(df.shape)

        MSSubClass  LotFrontage        LotArea  OverallQual  OverallCond  \
count  1460.000000  1201.000000    1460.000000  1460.000000  1460.000000   
mean     56.897260    70.049958   10516.828082     6.099315     5.575342   
std      42.300571    24.284752    9981.264932     1.382997     1.112799   
min      20.000000    21.000000    1300.000000     1.000000     1.000000   
25%      20.000000    59.000000    7553.500000     5.000000     5.000000   
50%      50.000000    69.000000    9478.500000     6.000000     5.000000   
75%      70.000000    80.000000   11601.500000     7.000000     6.000000   
max     190.000000   313.000000  215245.000000    10.000000     9.000000   

         YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1   BsmtFinSF2  ...  \
count  1460.000000   1460.000000  1452.000000  1460.000000  1460.000000  ...   
mean   1971.267808   1984.865753   103.685262   443.639726    46.549315  ...   
std      30.202904     20.645407   181.066207   456.098091   161.319273  ..

In [5]:
# drop series with similar stuff
# Shift param down to 0.95
# 1.00 = don't drop anything !!!
sameThresh = 1.00
dropSameList = []
for i in range(df.shape[1]):
    xsame = np.mean( df.iloc[0,i]==df.iloc[:,i])
    if(xsame>sameThresh):
        dropSameList.append(i) 
df.drop(df.columns[[dropSameList]],axis=1,inplace=True)

In [6]:
# Check NA's in each column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
drop_col = missing_value_df[missing_value_df.percent_missing > 50]
missing_list = drop_col.column_name.values
df.drop(missing_list,axis=1,inplace=True)

In [7]:
# generate dummies
df = pd.get_dummies(df)
# fill na's with means
df = df.fillna(df.mean())
print(df.describe())

        MSSubClass  LotFrontage        LotArea  OverallQual  OverallCond  \
count  1460.000000  1460.000000    1460.000000  1460.000000  1460.000000   
mean     56.897260    70.049958   10516.828082     6.099315     5.575342   
std      42.300571    22.024023    9981.264932     1.382997     1.112799   
min      20.000000    21.000000    1300.000000     1.000000     1.000000   
25%      20.000000    60.000000    7553.500000     5.000000     5.000000   
50%      50.000000    70.049958    9478.500000     6.000000     5.000000   
75%      70.000000    79.000000   11601.500000     7.000000     6.000000   
max     190.000000   313.000000  215245.000000    10.000000     9.000000   

         YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1   BsmtFinSF2  ...  \
count  1460.000000   1460.000000  1460.000000  1460.000000  1460.000000  ...   
mean   1971.267808   1984.865753   103.685262   443.639726    46.549315  ...   
std      30.202904     20.645407   180.569112   456.098091   161.319273  ..

In [8]:
#change y to SalesPrice log 
# y = np.log(df.SalePrice)
y = df.SalePrice

#drop original dependent variable
X = df.drop(['SalePrice'], axis=1)

In [9]:
df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


In [10]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0


For this problem you should download the KaggleAmes.csv data set from the data portion of the Latte website.  This is a large data set with house prices and lots of features variables.  Also, download the Jupyter notebook below which loads the data and does some initial preprocessing and cleaning.  It will eventually put the data you should use in the X and y variables as usual. For grid searches use the gridSearchCV() function.
1. Run a linear regression on the full sample, and report the R-squared value as you might have done in a standard econometrics class.
2. Now run the usual train/test split monte-carlo with half the sample in the test set.  Do this for 100 monte-carlo iterations, and report the mean R-squared on the train and test sets.
3. Repeat the same experiment (test/train split monte-carlo) for a small Ridge regression grid search.  Just use alpha = [0.5,1.,2.].  How is it doing compared to regular least squares on test data?
4. Repeat this, but this time add the StandardScaler() to your data preprocessing.  Did your results change?

In [11]:
#1.
lr = LinearRegression()
lr.fit(X,y)
print("Full sample R-squared:")
print(lr.score(X,y))

Full sample R-squared:
0.9313624499887028


In [12]:
#2.
def MCtraintest(nmc,X,y,modelObj,testFrac):
    # make space to store scores
    trainScore = np.zeros(nmc)
    testScore  = np.zeros(nmc)
    for i in range(nmc):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=testFrac)
        modelObj.fit(X_train,y_train)
        trainScore[i] = modelObj.score(X_train,y_train)
        testScore[i]  = modelObj.score(X_test,y_test)
    return trainScore,testScore

In [13]:
trainS, testS = MCtraintest(100,X,y,lr,0.5)
print("Train Score=",np.mean(trainS))
print("Test Score=",np.mean(testS))

Train Score= 0.9496383835487886
Test Score= -21023213.092998773


In [14]:
#3.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

score_used = 'r2' 
param_grid={'alpha':[0.5,1.,2.]}
shuffle_split = ShuffleSplit(test_size=0.5, n_splits=100)
grid_search=GridSearchCV(Ridge(),param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True)
grid_search.fit(X,y)
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_alpha']])

#Compared to the monte carlo simulation scores, the ridge regression seems to be doing much better.
#The test score for Monte Carlo is -21023213.092998773 which shows that this model is overfitted
#After ridge regression, the test scores are shown below. They are much better

   rank_test_score  mean_test_score param_alpha
0                3         0.771688         0.5
1                2         0.775871           1
2                1         0.779921           2


In [15]:
#Splitting the data into training and testing and then printing out the scores

X_train, X_test, y_train, y_test = train_test_split( X, y )
lr.fit(X_train,y_train)
print('Train Score for Linear Regression=',lr.score(X_train,y_train))
print('Test Score for Linear Regression=',lr.score(X_test,y_test))

#After splitting the model into training and testing data and performing linear regression, it can be seen the test score is 0.823
#The test Ridge regression results are slightly lower(0.771,0.775,0.779) compared to the test regression score after splitting the data (0.823)

Train Score for Linear Regression= 0.9423880620219615
Test Score for Linear Regression= 0.823418320290348


In [16]:
#4.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

fullModel = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])

# set up dictionary for grid search
param_grid={'ridge__alpha':[0.5,1.,2.]}
# set up cross-validation shuffles
shuffle_split = ShuffleSplit(test_size=0.5, n_splits=100)
# set up search
grid_search=GridSearchCV(fullModel,param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True,n_jobs=1)
# implement search
grid_search.fit(X,y)
# move results into DataFrame
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_ridge__alpha']])

#The test values after StandardScaler have improved slightly compared to the Ridge Regression model

   rank_test_score  mean_test_score param_ridge__alpha
0                3         0.781877                0.5
1                2         0.782923                  1
2                1         0.784711                  2
